***Importation des Bibliothéques***

In [ ]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

In [ ]:
DataSet = pd.read_csv('C:/Users/ASUS/Desktop/DataSet/DataSet.csv', header=0, index_col=0)

In [ ]:
DataSet= DataSet.astype(int)

In [ ]:
DataSet.to_csv('C:/Users/ASUS/Desktop/DataSet/DataSet_int.csv')

In [ ]:
DataSet.shape

(50315, 12741)

In [ ]:
counts = DataSet.sum()
# Créer un nouveau dataframe pour le tableau
Columns_sum = pd.DataFrame({"Nom de l'action": DataSet.columns, 'Nombre d\'utilisations': counts})
Columns_sum = Columns_sum.sort_values('Nombre d\'utilisations', ascending=False)

In [ ]:
Columns_sum

,Nom de l'action,Nombre d'utilisations
actions/checkout@v2,actions/checkout@v2,28284
actions/checkout@v3,actions/checkout@v3,16969
actions/setup-node@v1,actions/setup-node@v1,5373
actions/checkout@v1,actions/checkout@v1,4202
actions/setup-node@v3,actions/setup-node@v3,4166
...,...,...
docker://pointnetwork/pointnetwork_node:v0.1.83,docker://pointnetwork/pointnetwork_node:v0.1.83,1
docker://pipelinecomponents/remark-lint:latest,docker://pipelinecomponents/remark-lint:latest,1
docker://phparkitect/arkitect-github-actions:latest,docker://phparkitect/arkitect-github-actions:l...,1
docker://peaceiris/gh-pages:v2,docker://peaceiris/gh-pages:v2,1


***Prétraitement du DataSet***

In [ ]:
DataSet = DataSet.filter(regex='^(?!\.|docker://)')

In [ ]:
DataSet = DataSet.filter(regex='^(?!.*\.github).*')

In [ ]:
DataSet.to_csv('C:/Users/ASUS/Desktop/DataSet/DataSet_Pretraitee.csv')

In [ ]:
DataSet.shape

(50315, 11418)

In [ ]:
counts = DataSet.sum()
# Créer un nouveau dataframe pour le tableau
Columns_sum = pd.DataFrame({"Nom de l'action": DataSet.columns, 'Nombre d\'utilisations': counts})
Columns_sum = Columns_sum.sort_values('Nombre d\'utilisations', ascending=False)

In [ ]:
Columns_sum

,Nom de l'action,Nombre d'utilisations
actions/checkout@v2,actions/checkout@v2,28284
actions/checkout@v3,actions/checkout@v3,16969
actions/setup-node@v1,actions/setup-node@v1,5373
actions/checkout@v1,actions/checkout@v1,4202
actions/setup-node@v3,actions/setup-node@v3,4166
...,...,...
frenck/action-setup-cas@v0.1.1,frenck/action-setup-cas@v0.1.1,1
frenck/action-addon-linter@v2.6,frenck/action-addon-linter@v2.6,1
frenck/action-addon-linter@v2.5,frenck/action-addon-linter@v2.5,1
frenck/action-addon-linter@v2.11.0,frenck/action-addon-linter@v2.11.0,1


***DataSet Sans les Version des actions***

In [ ]:
DataSet.columns = DataSet.columns.str.split('/').str[1]
DataSet.columns = DataSet.columns.str.split('@').str[0]

In [ ]:
DataSet = DataSet.groupby(DataSet.columns, axis=1).max()

In [ ]:
DataSet.shape

(50315, 4444)

In [ ]:
counts = DataSet.sum()
# Créer un nouveau dataframe pour le tableau
Columns_sum = pd.DataFrame({"Nom de l'action": DataSet.columns, 'Nombre d\'utilisations': counts})
Columns_sum = Columns_sum.sort_values('Nombre d\'utilisations', ascending=False)

In [ ]:
Columns_sum

,Nom de l'action,Nombre d'utilisations
checkout,checkout,48378
setup-node,setup-node,12704
setup-python,setup-python,8026
cache,cache,7607
upload-artifact,upload-artifact,7456
...,...,...
github-action-testkube,github-action-testkube,1
github-action-validator,github-action-validator,1
github-action-version-branch,github-action-version-branch,1
github-action-version-pr,github-action-version-pr,1


In [ ]:
DataSet.to_csv('C:/Users/ASUS/Desktop/DataSet/DataSet_Sans_Version.csv')

***Extraction des actions***

In [ ]:
actions = DataSet.columns.tolist()

In [ ]:
GH_URL = 'https://github.com/marketplace/actions/{action}'

# Path to get the name of the action. Text content
NAME_PATH = 'h1.f1'
# Function to get the repository name
REPO_PATH = lambda s: s.find('h5', text='Links').find_next_sibling('a').text.strip()
# Path to get a list of categories. Text content (one or two items)
CATEGORY_PATH = '.topic-tag'
# Path to get the number of stars. Text content
STARS_PATH = '.js-social-count'
# Path to get versions. Text content (multiple items)
VERSION_PATH = '.select-menu-item-heading'

In [ ]:
output = []
done=[]

In [ ]:
def extract_html(action):
    # Check file exists
    r = requests.get(GH_URL.format(action=action))
    if r.status_code == 404:
        return None

    output = dict()
    soup = BeautifulSoup(r.text, 'html.parser')

    output['action'] = action

    name_elements = soup.select(NAME_PATH)
    if name_elements:
        output['name'] = name_elements[0].string

    repo_element = soup.find('h5', string='Links')
    if repo_element:
        output['repository'] = repo_element.find_next_sibling('a').text.strip()

    categories = soup.select(CATEGORY_PATH)
    output['category'] = categories[0].text.strip() if len(categories) > 0 else None
    output['category_secondary'] = categories[1].text.strip() if len(categories) == 2 else None

    stars_elements = soup.select(STARS_PATH)
    if stars_elements:
        output['stars'] = int(''.join(d for d in stars_elements[0]['title'] if d.isdigit())) if len(stars_elements[0]['title']) > 0 else 0

    output['versions'] = [e.text for e in soup.select(VERSION_PATH)]

    return output

In [ ]:
inputs = [a for a in actions if a not in done]

for action in tqdm(inputs):
    result = extract_html(action)

    if result is not None:
        output.append(result)

    done.append(action)

100%|██████████| 4444/4444 [30:07<00:00,  2.46it/s]


In [ ]:
print(f'{len(output)} actions found out of {len(actions)}.')

1404 actions found out of 4444.


In [ ]:
df_actions = (
    pd.DataFrame(output)
    .set_index('name')
    [['action', 'repository', 'category', 'category_secondary', 'stars', 'versions']]
)

In [ ]:
df_actions

,action,repository,category,category_secondary,stars,versions
name,,,,,,
1kWords,1kWords,loadmill/1kWords,Code quality,Utilities,1,"[v1.0.6, v1.0.5, v1.0.4, v1.0.2, v1.0.1]"
Badgr Create Badge,Badgr-Create-Badge,yajushiSri/Badgr-Create-Badge,Project management,Utilities,0,[v1.0.0]
ChatGPT Reviewer,ChatGPT-Reviewer,feiskyer/ChatGPT-Reviewer,Code review,AI Assisted,44,"[v0.6, v0.5, v0.4, v0.3, v0.2, v0.1]"
Checkout,Checkout,actions/checkout,Utilities,None,3988,"[v3.5.2, v3.5.1, v2.7.0, v3.5.0, v3.4.0, v3.3...."
DreamAnnotate,DreamAnnotate,yogstation13/DreamAnnotate,Continuous integration,Code quality,1,"[v2, v2-rc1, v1]"
...,...,...,...,...,...,...
Yarn Lock Changes,yarn-lock-changes,Simek/yarn-lock-changes,Code review,Dependency management,125,"[v0.11.1, v0.11.0, v0.10.0, v0.9.0, v0.8.1, v0..."
YC GitHub Runner,yc-github-runner,yc-actions/yc-github-runner,Continuous integration,Container CI,6,"[v1.1.0, v1.0.0]"
Zip npm files,zip-npm-files,Roslovets-Inc/zip-npm-files,Deployment,Utilities,4,[v1.0.0]


In [ ]:
df_actions.to_csv('C:/Users/ASUS/Desktop/DataSet/Actions.csv')